In [1]:
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

from qiskit.algorithms import VQE
from qiskit.circuit.library import TwoLocal
from qiskit.utils import QuantumInstance
from qiskit import Aer
from qiskit.algorithms.optimizers import COBYLA

import qiskit_alt, qiskit_alt.electronic_structure
from qiskit_alt.pauli_operators import PauliSum_to_SparsePauliOp
from qiskit.opflow import PauliSumOp

<h1> Describing the $H_2$ molecule structure </h1>

In [2]:
geometry = [["H", [0.0, 0.0, 0.0]], ["H", [0.0, 0.0, 0.735]]]
basis="sto3g"

molecule = Molecule(
    geometry=geometry, charge=0, multiplicity=1
)
driver = ElectronicStructureMoleculeDriver(
    molecule, basis=basis, driver_type=ElectronicStructureDriverType.PYSCF
)

<h1> Computing the fermionic operator </h1>

<h3> Qiskit Nature </h3>

In [3]:
es_problem = ElectronicStructureProblem(driver)
second_q_op = es_problem.second_q_ops()
qubit_converter = QubitConverter(mapper=JordanWignerMapper())
qubit_op = qubit_converter.convert(second_q_op[0])
qubit_op.primitive

SparsePauliOp(['IIII', 'ZIII', 'IZII', 'ZZII', 'IIZI', 'ZIZI', 'IZZI', 'IIIZ', 'ZIIZ', 'IZIZ', 'IIZZ', 'XXXX', 'YYXX', 'XXYY', 'YYYY'],
              coeffs=[-0.81054798+0.j, -0.22575349+0.j,  0.17218393+0.j,  0.12091263+0.j,
 -0.22575349+0.j,  0.17464343+0.j,  0.16614543+0.j,  0.17218393+0.j,
  0.16614543+0.j,  0.16892754+0.j,  0.12091263+0.j,  0.0452328 +0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j])

<h3> Qiskit Alt </h3>

In [8]:
# Compute the Fermionic operator of the molecule 
fermi_op = qiskit_alt.electronic_structure.fermionic_hamiltonian(geometry, basis)

# Convert the Fermionic operator to a Pauli operator using the Jordan-Wigner transform
pauli_op = qiskit_alt.electronic_structure.jordan_wigner(fermi_op);

# Convert the Pauli operator into a sum of Pauli operators
pauli_sum_op = PauliSumOp(pauli_op)

# Print the PauliSumOp operator, which will be the input to the VQE algorithm to compute the minimum eigenvalue
print(pauli_sum_op)

# Print the SparsePauliOp operator ~ Fermionic operator computed with qiskit-alt
pauli_op.simplify()

-0.090578986088348 * IIII
- 0.22575349222402383 * ZIII
+ 0.17218393261915532 * IZII
+ 0.12091263261776633 * ZZII
- 0.22575349222402383 * IIZI
+ 0.17464343068300456 * ZIZI
+ 0.16614543256382416 * IZZI
+ 0.04523279994605783 * XXXX
+ 0.04523279994605783 * YYXX
+ 0.04523279994605783 * XXYY
+ 0.04523279994605783 * YYYY
+ 0.1721839326191553 * IIIZ
+ 0.16614543256382416 * ZIIZ
+ 0.16892753870087907 * IZIZ
+ 0.12091263261776633 * IIZZ


SparsePauliOp(['IIII', 'ZIII', 'IZII', 'ZZII', 'IIZI', 'ZIZI', 'IZZI', 'IIIZ', 'ZIIZ', 'IZIZ', 'IIZZ', 'XXXX', 'YYXX', 'XXYY', 'YYYY'],
              coeffs=[-0.09057899+0.j, -0.22575349+0.j,  0.17218393+0.j,  0.12091263+0.j,
 -0.22575349+0.j,  0.17464343+0.j,  0.16614543+0.j,  0.17218393+0.j,
  0.16614543+0.j,  0.16892754+0.j,  0.12091263+0.j,  0.0452328 +0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j])

<h1> Computing the ground state energy </h1>

In [5]:
# Define the ansatz
ansatz = TwoLocal(
    rotation_blocks=["h", "rx"],
    entanglement_blocks="cz",
    entanglement="full",
    reps=2,
    parameter_prefix="y",
)

# Initialize the COBYLA optimizer
optimizer = COBYLA(maxiter=500)

# Select the backend for the quantum_instance
backend = QuantumInstance(Aer.get_backend('statevector_simulator'), 
                                        seed_simulator=100, 
                                        seed_transpiler=100)
# Run VQE algorithm
vqe = VQE(ansatz = ansatz, 
          optimizer = optimizer,
          quantum_instance = backend)

<h3> Qiskit Nature </h3>

In [6]:
# Compute the ground-state energy of the molecule 
result = vqe.compute_minimum_eigenvalue(operator=qubit_op)
print("The ground-state energy of the Hydrogen molecule is {} Hatree".format(round(result.eigenvalue.real,3)))

The ground-state energy of the Hydrogen molecule is -1.837 Hatree


<h3> Qiskit Alt </h3>

In [7]:
# Compute the ground-state energy of the molecule
result = vqe.compute_minimum_eigenvalue(operator=pauli_sum_op)
print("The ground-state energy of the Hydrogen molecule is {} Hatree".format(round(result.eigenvalue.real,3)))

The ground-state energy of the Hydrogen molecule is -1.117 Hatree
